 # Census cleanup 
To obtain the migration matrix, we need census data from 2013 that is compatible with the effective distance data obtained. 
This notebook converts a census dataset into a compatible format to be usable to calculate the migration matrix.

In [1]:
import pandas as pd
import pycountry

population_sizes = pd.read_csv('data_sources/population2013.csv', header=0)
population_sizes.shape

(271, 16)


Source: https://data-explorer.oecd.org/vis?tenant=archive&df[ds]=DisseminateArchiveDMZ&df[id]=DF_EDU_DEM&df[ag]=OECD&dq=..&pd=2013%2C2013&to[TIME_PERIOD]=false

Country populations in 2013 (since flight data was from 2013 and so fluxes rely on population structure from 2013)

In [2]:
population_sizes.head()

,Series Name,Series Code,Country Name,Country Code,1990 [YR1990],2000 [YR2000],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,"Population, total",SP.POP.TOTL,Afghanistan,AFG,10694796,19542982,32716210,33753499,34636207,35643418,36686784,37769499,38972230,40099462,41128771,42239854
1,"Population, total",SP.POP.TOTL,Albania,ALB,3286542,3089027,2889104,2880703,2876101,2873457,2866376,2854191,2837849,2811666,2777689,2745972
2,"Population, total",SP.POP.TOTL,Algeria,DZA,25518074,30774621,38760168,39543154,40339329,41136546,41927007,42705368,43451666,44177969,44903225,45606480
3,"Population, total",SP.POP.TOTL,American Samoa,ASM,47818,58230,52217,51368,50448,49463,48424,47321,46189,45035,44273,43914
4,"Population, total",SP.POP.TOTL,Andorra,AND,53569,66097,71621,71746,72540,73837,75013,76343,77700,79034,79824,80088


### Clean

In [3]:
population_sizes = population_sizes[['Country Code', '2014 [YR2014]', 'Country Name']]
population_sizes.reset_index(drop=True, inplace=True)
population_sizes.rename(columns={'Country Code': 'alpha3', '2014 [YR2014]':'population', 'Country Name':'fullname'}, inplace=True)
population_sizes.head()

,alpha3,population,fullname
0,AFG,32716210,Afghanistan
1,ALB,2889104,Albania
2,DZA,38760168,Algeria
3,ASM,52217,American Samoa
4,AND,71621,Andorra


## Replace alpha3 with alpha2


In [4]:
df = pd.DataFrame(columns=['alpha2', 'population', 'fullname'])
new_index = 0
for index, row in population_sizes.iterrows():
    try: 
        country = pycountry.countries.get(alpha_3 = row.alpha3)
        if country is not None: 
            df.loc[new_index] = [country.alpha_2] + [row.population] + [row.fullname]
            new_index += 1
    except:
        print(f'{row.alpha3} country not found')

nan country not found
nan country not found
nan country not found
nan country not found
nan country not found


In [5]:
df

,alpha2,population,fullname
0,AF,32716210,Afghanistan
1,AL,2889104,Albania
2,DZ,38760168,Algeria
3,AS,52217,American Samoa
4,AD,71621,Andorra
...,...,...,...
210,VI,107882,Virgin Islands (U.S.)
211,PS,4173398,West Bank and Gaza
212,YE,27753304,"Yemen, Rep."
213,ZM,15737793,Zambia


### Save result

In [7]:
df.to_csv(r'data_sources/census_2013.csv', index=None, sep=',', mode='w')
